In [0]:
import os
import numpy as np
import pandas as pd
import cv2
from zipfile import ZipFile 

file_name = '/9d34462453e311ea.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()  

In [4]:
 path = '/content/dataset'

 train_df = pd.read_csv(path+'/train.csv')
 test_df = pd.read_csv(path+'/test.csv')

 train_df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [5]:
class_map = {
    'Food': 0,
    'Attire': 1,
    'Decorationandsignage': 2,
    'misc': 3
}

inverse_class_map = {
    0: 'Food',
    1: 'Attire',
    2: 'Decorationandsignage',
    3: 'misc'
}

train_df['Class'] = train_df['Class'].map(class_map).astype(np.uint8)
train_df.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [0]:
h, w = 224, 224

In [0]:
train_path = '/content/dataset/Train Images/'
test_path = '/content/dataset/Test Images/'

train_images, train_labels = [], []

for i in range(len(train_df.Image)):
  train_image = cv2.imread(train_path + str(train_df.Image[i]))
  train_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2RGB)
  train_image = cv2.resize(train_image, (h, w))
  train_images.append(train_image)
  train_labels.append(train_df.Class[i])

test_images = []

for i in range(len(test_df.Image)):
  test_image = cv2.imread(test_path + str(test_df.Image[i]))
  test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
  test_image = cv2.resize(test_image, (h, w))
  test_images.append(test_image)

train_images = np.array(train_images)
test_images = np.array(test_images)

In [8]:
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_images, to_categorical(train_labels), test_size=0.3, random_state=42)

In [10]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(h, w, 3), 
    pooling='avg'
)

base_model.summary()













94658560/94653016 [==============================] - 7s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________

In [16]:
base_model.trainable = False

model = Sequential([
  base_model,
  # Dropout(0.2),
  Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
batch_size = 32
epochs = 10

In [0]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(monitor='val_loss', patience=3)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [17]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size), validation_data = (X_test, y_test),
                    steps_per_epoch = len(X_train) / batch_size, epochs = epochs, callbacks = callbacks)

# training_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='training',seed=7)
# validation_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='validation',seed=7)

# model.fit_generator(
#         train_generator,
#         steps_per_epoch=200,
#         epochs= epochs,
#         validation_data= validation_generator,
#         validation_steps= 80)

Epoch 1/10
131/130 [==============================] - 709s 5s/step - loss: 0.9214 - acc: 0.6315 - val_loss: 0.8648 - val_acc: 0.6886
Epoch 2/10
131/130 [==============================] - 714s 5s/step - loss: 0.6988 - acc: 0.7274 - val_loss: 0.7882 - val_acc: 0.7181
Epoch 3/10
131/130 [==============================] - 720s 5s/step - loss: 0.6443 - acc: 0.7631 - val_loss: 0.7929 - val_acc: 0.7036
Epoch 4/10
131/130 [==============================] - 732s 6s/step - loss: 0.5960 - acc: 0.7761 - val_loss: 0.9263 - val_acc: 0.6791

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/10
131/130 [==============================] - 742s 6s/step - loss: 0.5564 - acc: 0.7899 - val_loss: 0.8510 - val_acc: 0.7003


In [18]:
labels = model.predict(test_images)
print(labels[:4])

[[4.3809316e-01 4.6096030e-01 7.5737940e-04 1.0018913e-01]
 [9.3905401e-01 7.1608620e-03 2.2802607e-05 5.3762365e-02]
 [7.4877053e-01 1.6780206e-03 2.2154867e-04 2.4932992e-01]
 [9.9108726e-01 1.1264841e-05 1.3371615e-05 8.8881543e-03]]


In [19]:
label = [np.argmax(i) for i in labels]
print(label[:20])

[1, 0, 0, 0, 1, 3, 3, 3, 3, 1, 0, 2, 3, 2, 0, 0, 1, 3, 3, 1]


In [20]:
class_label = [inverse_class_map[x] for x in label]
print(class_label[:20])

['Attire', 'Food', 'Food', 'Food', 'Attire', 'misc', 'misc', 'misc', 'misc', 'Attire', 'Food', 'Decorationandsignage', 'misc', 'Decorationandsignage', 'Food', 'Food', 'Attire', 'misc', 'misc', 'Attire']


In [21]:
submission = pd.DataFrame({ 'Image': test_df.Image, 'Class': class_label })
submission.head()

,Image,Class
0,image6245.jpg,Attire
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [0]:
submission.to_csv('sub.csv', index=False)